In [ ]:
from __future__ import print_function

"""
A script to run cqed-rhf calculation of pyrimidine in the def2-TZVP basis
"""

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from matplotlib import pyplot as plt

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2


In [ ]:

# geometry for pyrimidine from Ryan's ORCA calculation
molstr = """

0 1
C     -3.138301    1.766784    0.005631
C     -2.051429    2.626262   -0.001165
C     -0.789779    2.053301   -0.001873
N     -0.631660    0.734101    0.003549
C     -1.729565   -0.011689    0.008655
N     -2.974582    0.448259    0.009994
H     -4.159547    2.136475    0.004056
H     -2.182537    3.700840   -0.006430
H      0.112609    2.657688   -0.007194
H     -1.597993   -1.090027    0.012382
no_reorient
symmetry c1
"""

# options dict
#options_dict = {'basis': 'def2-TZVP',
#               'save_jk': True, 
#               'scf_type': 'pk',
#               'e_convergence' : 1e-8,
#               'd_convergence' : 1e-7}

options_dict = {'basis': 'def2-TZVP',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-8,
               'd_convergence' : 1e-7}



Fundamental coupling strength, $\lambda$, is defined by the cavity volume $V$ as follows in atomic units:
$$ \lambda = \sqrt{\frac{4\pi}{V} } $$

Two ways of thinking about V:

Approach a:

$$ V_a = d^3, $$
where $d$ is the spacing between the gold nanoplates, which we can start by assuming is 2.4 $nm$ based on the geometry given [here](https://www.nature.com/articles/ncomms11749).

Approach b:

$$ V_b = d \cdot \lambda_p^2, $$
where $\lambda_p$ indicates the wavelength of the photon trapped, which we can assume is 240 $nm$ or 300 $nm$, based on pyrimidine's UV absorbance, as seen [here](https://www.sciencedirect.com/science/article/pii/S0009261420303845).  

Note that the distances here ($d$ and $\lambda_p$) are in nanometers, and they must be converted to atomic units before calculating $\lambda$.


In [ ]:
def nanometers_to_atomic_units(length_nm):
    ''' a function to convert a length in nanometers to a length in atomic units
    
    Arguments
    ---------
    length_nm : float
        a length in nanometers
        
    Returns
    -------
    length_au : float
       a length in atomic units
       
    Note: The atomic unit of length is often called the Bohr radius, so we need to have a
    conversion between 1 nanometer and X Bohr radii
       
    '''
    # <== do conversion here
    return length_au

def compute_coupling_approach_a(distance_au):
    ''' a function to take the spacing between cavity plates and return the fundamental coupling strength 
    
    Arguments
    ---------
    distance_au : float
        the distance between cavity plates in atomic units
        
    Returns
    -------
    lambda_au : float
        the fundamental coupling strength in atomic units
        
    Notes
    -----
    lambda = \sqrt(4\pi/V) where V = d^3 in this model
    
    '''
    # <== compute V here
    # <== compute lambda_au from V here!
    return lambda_au

def compute_coupling_approach_b(distance_au, lambda_p_au):
    ''' a function to take the spacing between cavity plate and the wavelength of a 
        resonant photon and return the fundamental coupling strength 
    
    Arguments
    ---------
    distance_au : float
        the distance between cavity plates in atomic units
        
    lambda_p_au : float
        the wavelength of the cavity photon in atomic units
        
    Returns
    -------
    lambda_au : float
        the fundamental coupling strength in atomic units
        
    Notes
    -----
    lambda = \sqrt(4\pi/V) where V = d * \lambda_p^2 in this model
    
    '''
    # <== compute V here
    # <== compute lambda_au from V here!
    return lambda_au



In [ ]:
# Use the functions that you completed above to compute the variable `lambda_val_a` and `lambda_val_b` to be used in 
# the subsequent cqed_rhf calculations

d_nm = 2.4 # spacing in nanometers

lambda_nm = 300 # wavelength in nanometers

# <== convert d_nm and lambda_nm to atomic units

# <== compute lambda_val_a through approach a

# <== compute lambda_val_b through approach b

Run the calculations below to compute the CQED-RHF energy for various polarizations and
without coupling to a photon.  

In [ ]:
# use the fundamental coupling strength to define the coupling vector
lam_y_a = np.array([0, lambda_val_a, 0])
lam_y_b = np.array([0, lambda_val_b, 0])

# define strings that will name the raw (unformatted) Fock matrix and overlap matrix files
fock_raw_string = 'F_raw.out'
overlap_raw_string = 'S_raw.out'

# open files with those names for writing
Fock_raw_file = open(fock_raw_string, 'w')
Overlap_raw_file = open(overlap_raw_string, 'w')


# Run the CQED-RHF calculation to get Fock and overlap matrices
cqed_dict_y = cqed_rhf(lam_y_a, molstr, options_dict)

# pull the Fock and overlap matrices from the results
F = cqed_dict_y["PF FOCK MATRIX"]
S = np.array(cqed_dict_y["OVERLAP MATRIX"])

# print the values of the Fock and Overlap matrices without formatting to the raw files named above
print(F'Fock Matrix for lambda_y = {l}', file=Fock_raw_file)
for i in range(0,len(F)):
    for j in range(i, len(F)):
        F_val = F[i,j]
        print(F' {i} {j} {F_val}', file=Fock_raw_file)
        
print(F'Overlap Matrix for lambda_y = {l}', file=Overlap_raw_file)
for i in range(0,len(S)):
    for j in range(i, len(S)):
        F_val = S[i,j]
        print(F' {i} {j} {F_val}', file=Overlap_raw_file)

# close the raw files - important so that the data actually gets written
Fock_raw_file.close()
Overlap_raw_file.close()

In [ ]:
# use numpy's loadtxt to read unformatted data and store in a numpy array
S_read = np.loadtxt(overlap_raw_string)
F_read = np.loadtxt(fock_raw_string)

# !!!! Change this string so that it contains information about the approach 
# !!!! for calculating coupling and the spacing, wavelength, and lambda value information
# e.g. if we use approach b with d = 2.4 nm and \lambda_p = 300 nm and that gives a lambda_val of 0.01 atomic units:
# formatted_string = 'approach_b_d_2.4_lambda_p_300_lambda_val_0.01.out'
# e.g. if we use approach a with d = 2.4 nm that gives lambda_val of 0.05:
# formatted_string = 'approach_a_d_2.4_lambda_val_0.05.out'

formatted_string = 'to_be_completed.out' # <== replace with appropriate string
formatted_fock_file = open(formatted_string, 'w')

In [ ]:
base = np.array([0, 1, 2, 3, 4, 5], dtype=int)
increment = np.array([6, 6, 6, 6, 6, 6], dtype=int)


In [ ]:
print('--------------', file=formatted_fock_file)
print('OVERLAP MATRIX', file=formatted_fock_file)
print('--------------', file=formatted_fock_file)
for j in range(35):
    idx = base + j * increment
    print(F' {idx[0]:13} {idx[1]:12} {idx[2]:12} {idx[3]:12} {idx[4]:12} {idx[5]:12}', file=formatted_fock_file )
    for i in range(0, 210):
        print(F' {i:4} {S[i, idx[0]]:12.6f} {S[i, idx[1]]:12.6f} {S[i, idx[2]]:12.6f} {S[i, idx[3]]:12.6f} {S[i, idx[4]]:12.6f} {S[i, idx[5]]:12.6f}', file=formatted_fock_file)

In [ ]:
print('Fock matrix for operator 0', file=formatted_fock_file)
for j in range(35):
    idx = base + j * increment
    print(F' {idx[0]:13} {idx[1]:12} {idx[2]:12} {idx[3]:12} {idx[4]:12} {idx[5]:12}', file=formatted_fock_file)
    for i in range(0, 210):
        print(F' {i:4} {F[i, idx[0]]:12.6f} {F[i, idx[1]]:12.6f} {F[i, idx[2]]:12.6f} {F[i, idx[3]]:12.6f} {F[i, idx[4]]:12.6f} {F[i, idx[5]]:12.6f}', file=formatted_fock_file)

In [ ]:
formatted_fock_file.close()